In [96]:
import pandas as pd
import numpy as np

df = pd.read_csv("airlines.csv")
# df_copy = df_copy.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
# df_copy.loc[df_copy.sample(frac=0.1).index, "Sex"] = np.nan
df.loc[df.sample(frac=0.1).index, "airportfrom"] = np.nan
df = df.drop(['flight'], axis=1)
df.isnull().sum()
nanInd = df.loc[pd.isna(df['airportfrom']), :].index
df
# df_orig.info()

,time,length,airline,airportfrom,airportto,dayofweek,delay
0,1235.0,80.0,MQ,DFW,CRP,5,0
1,1296.0,141.0,DL,ATL,HOU,1,0
2,360.0,146.0,OO,COS,ORD,4,0
3,1170.0,143.0,B6,BOS,CLT,3,0
4,1410.0,344.0,US,OGG,PHX,6,0
...,...,...,...,...,...,...,...
539378,530.0,72.0,OO,GEG,SEA,5,1
539379,560.0,115.0,WN,LAS,DEN,4,1
539380,827.0,74.0,EV,CAE,ATL,2,1
539381,715.0,65.0,WN,BWI,BUF,4,1


In [97]:
# cat_variables = df_copy[['Sex', 'Embarked']]
cat_variables = df[['airline', 'airportfrom', 'airportto']]
cat_dummies = pd.get_dummies(cat_variables, drop_first=True)
cat_dummies.loc[nanInd, cat_dummies.columns.str.startswith('airportfrom_')]= np.nan
cat_dummies

C:\Users\eilee\AppData\Local\Temp\ipykernel_1160\1637797334.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  cat_dummies.loc[nanInd, cat_dummies.columns.str.startswith('airportfrom_')]= np.nan


,airline_AA,airline_AS,airline_B6,airline_CO,airline_DL,airline_EV,airline_F9,airline_FL,airline_HA,airline_MQ,...,airportto_TXK,airportto_TYR,airportto_TYS,airportto_UTM,airportto_VLD,airportto_VPS,airportto_WRG,airportto_XNA,airportto_YAK,airportto_YUM
0,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539378,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539379,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539380,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539381,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [98]:
df = df.drop(['airline', 'airportfrom', 'airportto'], axis=1)
df = pd.concat([df, cat_dummies], axis=1)
# df

In [99]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
# df

c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future

In [100]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
# df

c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\eilee\anaconda3\envs\visual_inf\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future

KeyboardInterrupt: 

In [92]:
df.isna().sum()

time               0
length             0
dayofweek          0
delay              0
airline_B6         0
airline_CO         0
airline_DL         0
airline_FL         0
airline_MQ         0
airline_OO         0
airline_US         0
airline_WN         0
airportfrom_BMI    0
airportfrom_BWI    0
airportfrom_COS    0
airportfrom_DFW    0
airportfrom_EWR    0
airportfrom_MSY    0
airportfrom_OGG    0
airportto_BHM      0
airportto_CLT      0
airportto_CRP      0
airportto_DFW      0
airportto_GRR      0
airportto_HOU      0
airportto_MEM      0
airportto_ORD      0
airportto_PHX      0
dtype: int64

In [93]:
test = df.loc[nanInd]
test = test[test.columns[test.columns.str.startswith('airportfrom_')]]
test

,airportfrom_BMI,airportfrom_BWI,airportfrom_COS,airportfrom_DFW,airportfrom_EWR,airportfrom_MSY,airportfrom_OGG
3,0.2,0.0,0.0,0.4,0.2,0.0,0.0


In [95]:
imputed = df.drop(['time', 'length', 'dayofweek', 'delay'], axis=1)
imputed = imputed[imputed.columns[imputed.columns.str.startswith('airportfrom_')]]
imputed = imputed.loc[nanInd]
imputed['airport_from'] = imputed.idxmax(axis=1)
# imputed['airportfrom']='' # to create an empty column
# for col_name in imputed.columns:
#     imputed.loc[imputed[col_name]==1,'airportfrom']= imputed['airportfrom']+' '+col_name
imputed.head(10)

,airportfrom_BMI,airportfrom_BWI,airportfrom_COS,airportfrom_DFW,airportfrom_EWR,airportfrom_MSY,airportfrom_OGG,airport_from
3,0.2,0.0,0.0,0.4,0.2,0.0,0.0,airportfrom_DFW


In [162]:
# transforming original dataset
df_orig = pd.read_csv("titanic.csv")
df_orig = df_orig.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
cat_variables = df_orig[['Sex', 'Embarked']]
cat_dummies = pd.get_dummies(cat_variables, drop_first=True)
df_orig = df_orig.drop(['Sex', 'Embarked'], axis=1)
df_orig = pd.concat([df_orig, cat_dummies], axis=1)
df_orig.head()
scaler = MinMaxScaler()
df_orig = pd.DataFrame(scaler.fit_transform(df_orig), columns=df_orig.columns)

df_orig

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,0.0,1.0,0.271174,0.125,0.000000,0.014151,1.0,0.0,1.0
1,1.0,0.0,0.472229,0.125,0.000000,0.139136,0.0,0.0,0.0
2,1.0,1.0,0.321438,0.000,0.000000,0.015469,0.0,0.0,1.0
3,1.0,0.0,0.434531,0.125,0.000000,0.103644,0.0,0.0,1.0
4,0.0,1.0,0.434531,0.000,0.000000,0.015713,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
886,0.0,0.5,0.334004,0.000,0.000000,0.025374,1.0,0.0,1.0
887,1.0,0.0,0.233476,0.000,0.000000,0.058556,0.0,0.0,1.0
888,0.0,1.0,NaN,0.125,0.333333,0.045771,0.0,0.0,1.0
889,1.0,0.0,0.321438,0.000,0.000000,0.058556,1.0,0.0,0.0


In [164]:
comp = df_orig['Sex_male'].sub(df_copy['Sex_male'])
comp

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
886    0.0
887    0.0
888    0.0
889    0.0
890    0.0
Name: Sex_male, Length: 891, dtype: float64

In [165]:
comp.abs().mean()

0.058361391694725026

In [166]:
n = comp.size
(comp**2).sum()/n

0.058361391694725026

In [167]:
comp.sum()

52.0